In [ ]:
import pandas as pd

# Load the datasets
education_dataset = pd.read_csv('/mnt/data/data_wrangling_education_2024_u7568823.csv')
medical_dataset = pd.read_csv('/mnt/data/data_wrangling_medical_2024_u7568823.csv')

# Step 1: Merge the two datasets on the common attribute (SSN)
merged_dataset = pd.merge(education_dataset, medical_dataset, on='ssn', how='outer', suffixes=('_education', '_medical'))

# Step 2: Data Cleaning Tasks

# 1. Remove Duplicates based on 'ssn' and key attributes
merged_dataset.drop_duplicates(subset='ssn', keep='last', inplace=True)

# 2. Handle Missing Values
# For simplicity, let's fill categorical missing values with 'Unknown' and numerical missing values with the median.
categorical_cols = merged_dataset.select_dtypes(include=['object']).columns
numerical_cols = merged_dataset.select_dtypes(include=['number']).columns

# Fill missing values
merged_dataset[categorical_cols] = merged_dataset[categorical_cols].fillna('Unknown')
merged_dataset[numerical_cols] = merged_dataset[numerical_cols].fillna(merged_dataset[numerical_cols].median())

# 3. Detect and handle inconsistent values (e.g., negative values in 'salary' and 'weight')
merged_dataset['salary'] = merged_dataset['salary'].apply(lambda x: x if x >= 0 else merged_dataset['salary'].median())
merged_dataset['weight'] = merged_dataset['weight'].apply(lambda x: x if x >= 0 else merged_dataset['weight'].median())

# 4. Imputation for values where it makes sense
# For instance, calculate BMI from weight and height if BMI is missing
def calculate_bmi(row):
    if pd.isnull(row['bmi']) and not pd.isnull(row['weight']) and not pd.isnull(row['height']):
        height_in_meters = row['height'] / 100
        return row['weight'] / (height_in_meters ** 2)
    return row['bmi']

merged_dataset['bmi'] = merged_dataset.apply(calculate_bmi, axis=1)

# Step 5: Handle Invalid Email and Phone entries
# Detect phone numbers in the email field and vice versa
import re

def looks_like_phone(value):
    if pd.isnull(value):
        return False
    digits_only = re.sub(r'\D', '', str(value))  # Remove non-digit characters
    return len(digits_only) >= 10  # Assume phone numbers have at least 10 digits

def looks_like_email(value):
    if pd.isnull(value):
        return False
    return re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', str(value)) is not None

# Correct email and phone based on validation
merged_dataset.loc[merged_dataset['email'].apply(looks_like_phone), 'email'] = 'Invalid Email'
merged_dataset.loc[merged_dataset['phone'].apply(looks_like_email), 'phone'] = 'Invalid Phone'

# Output the cleaned dataset to a CSV
output_file = '/mnt/data/cleaned_merged_dataset.csv'
merged_dataset.to_csv(output_file, index=False)

output_file


output duplicate ssn

In [3]:
import pandas as pd

education_df = pd.read_csv('files/lastVersion/merged_education_medical.csv')

# Filter rows with duplicate SSNs
duplicate_ssn_education = education_df[education_df.duplicated(subset='ssn', keep=False)]

# Save the duplicate entries to a new CSV file
duplicate_ssn_education.to_csv('files/lastVersion/duplicate_mergeDataset.csv', index=False)


merge 2 data set

In [2]:
import pandas as pd

# Load the CSV files into dataframes
education_df = pd.read_csv('data_wrangling_education_2024_u7568823.csv')
medical_df = pd.read_csv('data_wrangling_medical_2024_u7568823.csv')

# Drop the 'rec_id' column from both datasets before merging
education_df_cleaned = education_df.drop(columns=['rec_id'])
medical_df_cleaned = medical_df.drop(columns=['rec_id'])

# Perform the merge based on the 'ssn' column, using an inner join
merged_df = pd.merge(education_df_cleaned, medical_df_cleaned, on='ssn', suffixes=('_education', '_medical'))

# Save the merged dataframe to a CSV file if needed
merged_df.to_csv('files/lastVersion/merged_education_medical.csv', index=False)

# Optionally, display the merged dataframe
print(merged_df.head())


          ssn first_name_education middle_name_education last_name_education  \
0  g180519599              matthew              mcfarrin             lovette   
1  e141846696                 john                   NaN              stultz   
2  h140232568               carrie                  hall              greene   
3  c129535181                diane                     j             gunther   
4  e145580195               joseph                 vance          fitzgerald   

  gender_education  current_age birth_date_education  \
0                m           77            5/12/1943   
1                m           16             6/9/2004   
2                f           30           17/10/1990   
3                f           34           25/11/1986   
4                m           54            23/3/1966   

               street_address_education suburb_education  postcode_education  \
0    36  zouch  street  edward  street             young              2594.0   
1           533  north

merge newer first_name,middle_name,last_name

In [6]:
import pandas as pd

merged_df = pd.read_csv('files/lastVersion/merged_education_medical.csv')

# First, convert the timestamp columns to datetime to enable comparison
merged_df['employment_timestamp'] = pd.to_datetime(merged_df['employment_timestamp'], errors='coerce')
merged_df['consultation_timestamp'] = pd.to_datetime(merged_df['consultation_timestamp'], errors='coerce')

# Define a function to keep the names based on the newer timestamp
def choose_newer_name(row):
    if pd.isnull(row['employment_timestamp']):
        return row['first_name_medical'], row['middle_name_medical'], row['last_name_medical']
    if pd.isnull(row['consultation_timestamp']):
        return row['first_name_education'], row['middle_name_education'], row['last_name_education']
    if row['consultation_timestamp'] > row['employment_timestamp']:
        return row['first_name_medical'], row['middle_name_medical'], row['last_name_medical']
    else:
        return row['first_name_education'], row['middle_name_education'], row['last_name_education']

# Apply the function to each row
merged_df[['first_name', 'middle_name', 'last_name']] = merged_df.apply(
    lambda row: pd.Series(choose_newer_name(row)), axis=1
)

# Drop the old columns with suffixes
merged_df.drop(columns=['first_name_education', 'first_name_medical', 
                        'middle_name_education', 'middle_name_medical', 
                        'last_name_education', 'last_name_medical'], inplace=True)

name_columns = ['first_name', 'middle_name', 'last_name']
columns_order = ['ssn'] + name_columns + [col for col in merged_df.columns if col not in name_columns and col != 'ssn']

# Reorder the columns
merged_df = merged_df[columns_order]

# Display the updated dataframe or save to a file if needed
merged_df.to_csv('files/lastVersion//merged_names_dataset.csv', index=False)


In [10]:

merged_df = pd.read_csv('files/lastVersion/merged_names_dataset.csv')

# Convert the timestamps to datetime format for comparison
merged_df['employment_timestamp'] = pd.to_datetime(merged_df['employment_timestamp'], errors='coerce')
merged_df['consultation_timestamp'] = pd.to_datetime(merged_df['consultation_timestamp'], errors='coerce')

# Merge common attributes based on the most recent timestamp

# Gender
merged_df['gender'] = merged_df.apply(
    lambda row: row['gender_education'] if row['employment_timestamp'] > row['consultation_timestamp'] else row['gender_education'], axis=1)

# Birth Date
merged_df['birth_date'] = merged_df.apply(
    lambda row: row['birth_date_education'] if row['employment_timestamp'] > row['consultation_timestamp'] else row['birth_date_education'], axis=1)

# Street Address
merged_df['street_address'] = merged_df.apply(
    lambda row: row['street_address_education'] if row['employment_timestamp'] > row['consultation_timestamp'] else row['street_address_education'], axis=1)

# Suburb
merged_df['suburb'] = merged_df.apply(
    lambda row: row['suburb_education'] if row['employment_timestamp'] > row['consultation_timestamp'] else row['suburb_education'], axis=1)

# Postcode
merged_df['postcode'] = merged_df.apply(
    lambda row: row['postcode_education'] if row['employment_timestamp'] > row['consultation_timestamp'] else row['postcode_education'], axis=1)

# State
merged_df['state'] = merged_df.apply(
    lambda row: row['state_education'] if row['employment_timestamp'] > row['consultation_timestamp'] else row['state_education'], axis=1)

# Drop the old columns with suffixes from both datasets
columns_to_drop = ['gender_education', 'birth_date_education', 'street_address_education', 
                   'suburb_education', 'postcode_education', 'state_education']
merged_df.drop(columns=columns_to_drop, inplace=True)

# Save the merged dataframe or display
merged_df.to_csv('files/lastVersion/merged_common_attributes.csv')


In [12]:
import pandas as pd

# Load the dataset (replace 'file_path' with your actual file path)
file_path = 'files/merged_dataset_Pro_maritalStatusAndSalary_imputation.csv'
data = pd.read_csv(file_path)

# Convert height from cm to meters
data['height_m'] = data['height'] / 100

# Identify rows where weight is negative
negative_weight_rows = data[data['weight'] < 0]

# Correct the negative weights using the formula: weight = BMI * (height_m)^2
data.loc[data['weight'] < 0, 'weight'] = data['bmi'] * (data['height_m'] ** 2)
# Round the weight to 1 decimal place
data['weight'] = data['weight'].round(1)
# Drop the temporary 'height_m' column after correction
data.drop(columns=['height_m'], inplace=True)

# Save the corrected dataset back to a CSV file (optional)
data.to_csv('merged_dataset_correctWeight.csv', index=False)

# Display the corrected rows (optional)
corrected_rows = data.loc[negative_weight_rows.index]
print(corrected_rows)


C:\Users\13521\AppData\Local\Temp\ipykernel_2780\1512346858.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[59.9781 63.368  51.84   ... 95.83   59.2704 24.48  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[data['weight'] < 0, 'weight'] = data['bmi'] * (data['height_m'] ** 2)


              ssn  age_at_consultation      medicare_number    marital_status  \
10     d165618464                   63   3242  66532  2  2            married   
35     i150642589                   21   3385  37618  2  3            married   
37     a145218863                   66   2112  80784  2  1            married   
40     e193947611                   63   3631  95123  2  1        not-married   
60     f199940301                   49   8951  39432  2  2        not-married   
...           ...                  ...                  ...               ...   
16764  g192756040                   51   2383  67740  2  1   married-de-facto   
16778  h131934275                   28   5102  69110  1  1   married-de-facto   
16785  e156647896                   19   2123  64651  1  2            married   
16786  c161075281                   24   2642  10963  1  2        not-married   
16789  g153285282                    9   1364  41872  1  3            married   

       height  weight  bmi 

In [15]:
import pandas as pd
new_data = pd.read_csv('files/finalMergeEmail.csv')
# new_data['email'] = new_data['email'].fillna('').astype(str)
# Define a regex pattern for a valid email address
email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'

# Check if each email matches the valid pattern
invalid_emails = new_data[~new_data['email'].str.match(email_pattern)]

# Display rows with invalid email addresses
print(invalid_emails)


              ssn  age_at_consultation      medicare_number    marital_status  \
6      i131945217                   29   4065  69117  1  2            married   
19     g197247139                   27   7250  38940  2  1        not-married   
27     d113054410                   30   6927  49080  1  2        not-married   
47     c126684873                   57   2908  17186  2  1   married-de-facto   
55     f166206248                   21   5065  40504  2  3        not-married   
...           ...                  ...                  ...               ...   
16764  g192756040                   51   2383  67740  2  1   married-de-facto   
16766  b164323387                   54   6004  97272  2  2            married   
16794  e155951170                   20   6298  84295  1  3        not-married   
16801  a184970069                   48   8214  95221  2  1           divorced   
16808  d136644076                    9   2399  36590  1  3        not-married   

       height  weight  bmi 

merge 2 dataset but only keep the correct email and ssn

In [17]:
# Load the two newly uploaded files
file_path_education = 'data_wrangling_education_2024_u7568823.csv'
file_path_medical = 'data_wrangling_medical_2024_u7568823.csv'

# Load both datasets
education_data = pd.read_csv(file_path_education)
medical_data = pd.read_csv(file_path_medical)

# Merge the datasets based on 'ssn' and keep only the 'email' column
merged_data = pd.merge(education_data[['ssn', 'email']], medical_data[['ssn', 'email']], on='ssn', suffixes=('_edu', '_med'))

# Apply the function to select the correct email format between the two
def choose_correct_email(row):
    email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    
    # Check if the education email is valid
    if pd.notnull(row['email_edu']) and re.match(email_pattern, row['email_edu']):
        return row['email_edu']
    
    # Otherwise, check if the medical email is valid
    if pd.notnull(row['email_med']) and re.match(email_pattern, row['email_med']):
        return row['email_med']
    
    # If both are invalid or missing, return NaN
    return None

# Apply the function to the merged data
merged_data['correct_email'] = merged_data.apply(choose_correct_email, axis=1)

# Drop the individual email columns and keep only the 'ssn' and 'correct_email'
final_data = merged_data[['ssn', 'correct_email']]
final_data.to_csv('files/correctEmail.csv', index=False)



replace the email

In [21]:
import pandas as pd


# Load the new uploaded files
file_path_correct_email = 'files/correctEmail.csv'
file_path_final_imputate_salary = 'files/finalImputateSalary.csv'


# Load both datasets
correct_email_data = pd.read_csv(file_path_correct_email)
final_imputate_salary_data = pd.read_csv(file_path_final_imputate_salary)

# Merge the datasets on 'ssn', replacing the 'email' in finalImputateSalary with the correct one from correctEmail.csv
updated_final_data = pd.merge(final_imputate_salary_data.drop(columns=['email'], errors='ignore'), 
                              correct_email_data[['ssn', 'correct_email']], 
                              on='ssn', 
                              how='left')

# Rename 'correct_email' to 'email' for consistency
updated_final_data.rename(columns={'correct_email': 'email'}, inplace=True)
updated_final_data['email'].fillna('missing_email@example.com', inplace=True)
updated_final_data.to_csv('files/finalMergeEmail.csv', index=False)



C:\Users\13521\AppData\Local\Temp\ipykernel_2780\3855944602.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  updated_final_data['email'].fillna('missing_email@example.com', inplace=True)
